In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import ignite
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import string
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
np.random.seed(123)
torch.manual_seed(123)
#https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/

Using TensorFlow backend.


In [2]:
df = pd.read_csv('PBP - 2016 - Week 1.csv')
df.head()

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description
0,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,15:00,2016-09-03T06:36:41Z,Kickoff,1,10,35,10,86,"Drew Galitz kickoff for 61 yds , Shakeir Ryan ..."
1,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,1,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Timeout,1,10,86,0,86,"Timeout NORTHWESTERN ST, clock 14:55"
2,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,2,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,1,10,86,-1,87,De'Mard Llorens run for a loss of 1 yard to th...
3,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,3,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,2,11,87,8,79,De'Mard Llorens run for 8 yds to the NWSt 21
4,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,4,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,3,3,79,0,79,De'Mard Llorens run for no gain to the NWSt 21


In [27]:
df.query('driveIndex == 1 & gameId == 400868876').iloc[[-1]]['type'].values[0]

'Rushing Touchdown'

### Make the Result Column

In [3]:
def getDriveResult(driveId, gameId, df):
    final = df.query('driveIndex == @driveId & gameId == @gameId').iloc[[-1]]
    return final['type'].values[0]

In [4]:
for index,row in df.iterrows():
    df.at[index,'result'] = getDriveResult(row['driveIndex'], row['gameId'], df)

In [5]:
df

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,result
0,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,15:00,2016-09-03T06:36:41Z,Kickoff,1,10,35,10,86,"Drew Galitz kickoff for 61 yds , Shakeir Ryan ...",Punt
1,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,1,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Timeout,1,10,86,0,86,"Timeout NORTHWESTERN ST, clock 14:55",Punt
2,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,2,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,1,10,86,-1,87,De'Mard Llorens run for a loss of 1 yard to th...,Punt
3,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,3,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,2,11,87,8,79,De'Mard Llorens run for 8 yds to the NWSt 21,Punt
4,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,4,2466,Northwestern St,NWST,239,Baylor,BAY,0,0,1,14:55,2016-09-03T06:38:11Z,Rush,3,3,79,0,79,De'Mard Llorens run for no gain to the NWSt 21,Punt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16136,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,0,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,1,10,33,15,48,B.J. Smith run for 15 yds to the Troy 48 for a...,End of Game
16137,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,1,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,1,10,48,3,51,B.J. Smith run for 3 yds to the APeay 49,End of Game
16138,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,2,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,2,7,51,-2,49,B.J. Smith run for a loss of 2 yards to the Tr...,End of Game
16139,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,3,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,3,9,49,8,57,B.J. Smith run for 8 yds to the APeay 43,End of Game


In [6]:
df['description'] = df['description'].str.replace('[^\w\s]','')
df['description'] = df['description'].str.replace('[{}]'.format(string.punctuation), '')
df['result'] = df['result'].str.replace('[^\w\s]','')

In [7]:
x_train = df['description']
y_train = df['result']

x_train = x_train.values
y_train = y_train.values

### Tokenization

In [8]:
## create tokens 
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [9]:
## ensure that all the data is actually text
new_x_train = []
for i in x_train:
    new_x_train.append(str(i))
    
x_train = new_x_train

In [10]:
## ensure that all the data is actually text
new_y_train = []
for i in y_train:
    new_y_train.append(str(i))
    
y_train = new_y_train

In [11]:
## convert texts to padded sequences 
x_train = tokenizer.texts_to_sequences(x_train)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen = 70)

In [12]:
## convert texts to padded sequences 
y_train = tokenizer.texts_to_sequences(y_train)
y_train = keras.preprocessing.sequence.pad_sequences(y_train, maxlen = 70)

In [13]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

In [14]:
EMBEDDING_FILE = 'glove.840B.300d.txt'

embeddings_index = {}
for i, line in enumerate(open(EMBEDDING_FILE, encoding='utf8')):
    val = line.split()
    embeddings_index[val[0]] = np.asarray(val[1:], dtype='float32')
    


ValueError: could not convert string to float: '.'

In [15]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        ## Embedding Layer, Add parameter 
        self.embedding = nn.Embedding(2000, embedding_matrix.size) 
        et = torch.tensor(embedding_matrix, dtype=torch.float32)
        self.embedding.weight = nn.Parameter(et)
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(300, 40)        
        self.linear = nn.Linear(40, df['result'].unique().size)
        self.out = nn.Linear(df['result'].unique().size, df['result'].unique().size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        h_embedding = self.embedding(x)        
        h_lstm, _ = self.lstm(h_embedding)
        max_pool, _ = torch.max(h_lstm, 1)        
        linear = self.relu(self.linear(max_pool))
        out = self.out(linear)
        return out
    
model = Model()

In [17]:
## create training and validation split 
split_size = int(0.8 * len(df))
index_list = list(range(len(df)))
train_idx, valid_idx = index_list[:split_size], index_list[split_size:]

In [171]:
for i in df['result'].unique():
    print(i)

Punt
Rushing Touchdown
Field Goal Good
Passing Touchdown
Pass Incompletion
End of Half
Fumble Recovery Opponent
End of Game
Field Goal Missed
Penalty
Pass Interception Return
Blocked Field Goal
Defensive 2pt Conversion
Rush
End Period
Pass Reception
Timeout
Sack
Interception Return Touchdown
Fumble Recovery Own
Kickoff
Kickoff Return Touchdown
Safety
Blocked Punt
Punt Return Touchdown
nan
Fumble Return Touchdown


In [22]:
## create iterator objects for train and valid datasets
x_tr = torch.tensor(x_train[train_idx], dtype=torch.long)
y_tr = torch.tensor(y_train[train_idx], dtype=torch.long)
train = TensorDataset(x_tr, y_tr)

trainloader = torch.utils.data.DataLoader(train, batch_size=1,shuffle=True)


In [19]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

In [27]:
for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            print(labels)
            # forward + backward + optimize
            outputs = model(inputs)
            print(outputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6, 27]])
tensor([[-0.0875, -0.1755, -0.0989,  0.1627,  0.0399,  0.2230, -0.0110,  0.0814,
          0.1872, -0.0475, -0.0871,  0.0793,  0.0993, -0.1597,  0.1058, -0.0977,
         -0.0971, -0.0134,  0.1487, -0.0686,  0.0188, -0.0728, -0.0832, -0.0270,
         -0.0203,  0.1696,  0.2627]], grad_fn=<AddmmBackward>)


RuntimeError: multi-target not supported at ../aten/src/THNN/generic/ClassNLLCriterion.c:22

In [247]:
softmax = nn.Softmax()

In [285]:
total_correct = 0
total_images = 0
preds = np.empty((0,1))
lebs = np.empty((0,1))
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels.long()).sum().item()
        preds = np.append(preds,predicted)
        lebs = np.append(lebs,labels)

print(total_correct)
model_accuracy = total_correct / total_images * 100
print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))

4618
Model accuracy on 12912 test images: 35.77%


In [272]:
softmax(model(x_tr[23].unsqueeze(0)))

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[3.3676e-01, 1.3001e-01, 1.1210e-01, 1.4262e-01, 2.3763e-02, 3.1304e-02,
         2.7400e-02, 3.1378e-02, 4.6385e-02, 5.2695e-03, 4.0285e-02, 6.2177e-03,
         7.8843e-04, 2.3580e-02, 7.7809e-03, 6.0888e-03, 1.9740e-03, 1.3885e-02,
         5.3413e-03, 1.8886e-03, 4.9410e-04, 2.9903e-04, 2.4516e-03, 1.4195e-03,
         4.9395e-04, 1.4249e-05, 1.4249e-05]], grad_fn=<SoftmaxBackward>)

In [273]:
for j in range(20):
    
    probs, index = torch.max(softmax(model(x_tr[i*79].unsqueeze(0))), dim=1)

    for p, i in zip(probs, index):
        print('{0} - {1:.4f}'.format(df['result'].unique()[i], p))

Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368
Punt - 0.3368


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until
